# [실습] LangChain Code Interpreter를 이용한 데이터 시각화

Python repl_tool은 파이썬 코드를 입력받아 로컬에서 실행하는 기능입니다.   
데이터 파일을 전달하고, 이를 위한 시각화를 수행해 보겠습니다.


In [ ]:
!pip install langgraph openai matplotlib langchain langchain-openai langchain-community langchain-experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


이번 실습은 한국어 시각화가 필요할 수 있으므로, 코랩에서는 아래 옵션을 실행합니다.

In [ ]:
!pip install koreanize_matplotlib

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv('env', override=True)

llm = ChatOpenAI(model = 'gpt-4.1-mini', max_tokens=1024, temperature=0.1)

Python REPL을 불러옵니다.    
이 클래스는 run을 통해 실행하는 구조로, 별도의 툴로 Wrap할 수 있습니다.

In [ ]:
from langchain_experimental.tools.python.tool import PythonREPL
repl = PythonREPL()

In [ ]:
example_code = "for i in range(10): print(i)"
repl.run(example_code)

'0\n1\n2\n3\n4\n5\n6\n7\n8\n9\n'

실제 구현에서의 툴은 아래와 같이 자세한 내용을 포함하는 것이 좋습니다.

In [ ]:
from langchain_core.tools import tool
from typing import Annotated
from langchain_experimental.tools.python.tool import PythonREPL
repl = PythonREPL()

example_code = "for i in range(10): print(i)"
repl.run(example_code)

# 툴 정보를 보다 자세하게 작성하여 성능을 높입니다.
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you MUST print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return result_str

python_repl_tool을 LLM에 binding합니다.

In [ ]:
tools = [python_repl_tool]

In [ ]:
from rich import print as rprint
llm_with_tools = llm.bind_tools(tools)
rprint(llm_with_tools)

RunnableBinding(
    bound=ChatOpenAI(
        client=<openai.resources.chat.completions.completions.Completions object at 0x79dbed578620>,
        async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x79dbecf55a90>,
        root_client=<openai.OpenAI object at 0x79dcd32647a0>,
        root_async_client=<openai.AsyncOpenAI object at 0x79dbed578650>,
        model_name='gpt-4.1-mini',
        temperature=0.1,
        model_kwargs={},
        openai_api_key=SecretStr('**********'),
        stream_usage=True,
        max_tokens=1024
    ),
    kwargs={
        'tools': [
            {
                'type': 'function',
                'function': {
                    'name': 'python_repl_tool',
                    'description': 'Use this to execute python code. If you want to see the output of a value,\n   
you MUST print it out with `print(...)`. This is visible to the user.',
                    'parameters': {
                        'properties': {'code': {'description': 'The python code to execute.', 'type': 'string'}},
                        'required': ['code'],
                        'type': 'object'
                    }
                }
            }
        ]
    },
    config={},
    config_factories=[]
)

예시 데이터를 전달해 보겠습니다.

In [ ]:
import pandas as pd
df = pd.read_csv('./sample_data.csv')
df.head()


,날짜,제품,가격,판매량,고객_연령,고객_성별,지역
0,2023-01-01,노트북,1200000,5,34,남성,서울
1,2023-01-02,노트북,1200000,3,42,여성,부산
2,2023-01-03,노트북,1200000,2,28,남성,대구
3,2023-01-04,노트북,1200000,7,45,여성,인천
4,2023-01-05,노트북,1200000,4,31,남성,광주


이전과 동일하게, LLM에게 메시지를 전달해 보겠습니다.   
Step-by-Step으로 작업을 수행하도록 구성합니다.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage('''당신은 파이썬 코드를 이용한 데이터 분석의 전문가입니다.
주어진 툴의 사용 방법을 참고하여 분석을 수행하세요.
문제를 Step by Step으로 분할하여 풀고, 중간 출력은 항상 Print를 해야 확인이 가능합니다.
한국어 시각화가 필요한 경우, import koreanize_matplotlib을 사용하시오. ''')

query = HumanMessage('파일의 위치가 ./sample_data.csv 이다. 이 파일의 제품별 판매량을 비교하는 시각화를 result.png에 저장해줘.')
msgs = [system_message, query]

In [ ]:
tool_call_msg = llm_with_tools.invoke(msgs)
rprint(tool_call_msg)

AIMessage(
    content='',
    additional_kwargs={
        'tool_calls': [
            {
                'id': 'call_H7eXlOODk5e9mvgFxUdVcysV',
                'function': {
                    'arguments': '{"code":"import pandas as pd\\nimport matplotlib.pyplot as plt\\nimport 
koreanize_matplotlib\\n\\n# 데이터 불러오기\\ndf = pd.read_csv(\'./sample_data.csv\')\\n\\n# 데이터 
확인\\nprint(df.head())"}',
                    'name': 'python_repl_tool'
                },
                'type': 'function'
            }
        ],
        'refusal': None
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 61,
            'prompt_tokens': 189,
            'total_tokens': 250,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-4.1-mini-2025-04-14',
        'system_fingerprint': 'fp_4c2851f862',
        'id': 'chatcmpl-CT1mfSRUU2irbhCDONPB71cJwb26w',
        'service_tier': 'default',
        'finish_reason': 'tool_calls',
        'logprobs': None
    },
    id='run--93b756a3-dace-4975-a8f0-41bf15ae3923-0',
    tool_calls=[
        {
            'name': 'python_repl_tool',
            'args': {
                'code': "import pandas as pd\nimport matplotlib.pyplot as plt\nimport koreanize_matplotlib\n\n# 
데이터 불러오기\ndf = pd.read_csv('./sample_data.csv')\n\n# 데이터 확인\nprint(df.head())"
            },
            'id': 'call_H7eXlOODk5e9mvgFxUdVcysV',
            'type': 'tool_call'
        }
    ],
    usage_metadata={
        'input_tokens': 189,
        'output_tokens': 61,
        'total_tokens': 250,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 0}
    }
)

In [ ]:
print(tool_call_msg.tool_calls[0]['args']['code'])

import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib

# 데이터 불러오기
df = pd.read_csv('./sample_data.csv')

# 데이터 확인
print(df.head())


In [ ]:
python_repl_tool.invoke(tool_call_msg.tool_calls[0])

ToolMessage(content="Successfully executed:\n```python\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport koreanize_matplotlib\n\n# 데이터 불러오기\ndf = pd.read_csv('./sample_data.csv')\n\n# 데이터 확인\nprint(df.head())\n```\nStdout:            날짜   제품       가격  판매량  고객_연령 고객_성별  지역\n0  2023-01-01  노트북  1200000    5     34    남성  서울\n1  2023-01-02  노트북  1200000    3     42    여성  부산\n2  2023-01-03  노트북  1200000    2     28    남성  대구\n3  2023-01-04  노트북  1200000    7     45    여성  인천\n4  2023-01-05  노트북  1200000    4     31    남성  광주\n", name='python_repl_tool', tool_call_id='call_H7eXlOODk5e9mvgFxUdVcysV')

위와 같이 데이터 분석과 같은 문제는 단순히 툴을 1회 실행하는 것이 아닌, 추가적인 작업이 필요할 수 있습니다.    
메시지의 Context를 계속 늘리며, 툴이 필요할 때마다 실행하도록 만들어 보겠습니다.

In [ ]:
import koreanize_matplotlib

def react_agent(llm, msgs , tools = [python_repl_tool]):

    # 툴과 LLM 구성

    tool_list = {x.name: x for x in tools}
    # tavily_search.name = 'tavily_search' 을 이용하면
    # tool_list = {'tavily_search': tavily_search} 와 동일합니다.

    llm_with_tools = llm.bind_tools(tools)


    # 메시지 구성
    # messages = [HumanMessage(question)]
    # print('Query:', question)
    messages = msgs


    # LLM에 메시지 전달 (분기)
    tool_msg = llm_with_tools.invoke(question)
    # AIMessage


    # 1) 툴이 필요없으면: 응답 후 종료
    # 2) 툴이 필요하면: AIMessage (+ Tool Call)
    print('## LLM 호출 ##')
    messages.append(tool_msg)
    # [Human, AI]

    if tool_msg.content:
        print('LLM:', tool_msg.content)

    while tool_msg.tool_calls:
        # 툴 호출이 있을 경우: 툴 실행 후 결과를 전달 (반복)

        for tool_call in tool_msg.tool_calls:
            tool_name = tool_call['name']

            print(f"-- {tool_name} 사용 중 --")
            print(tool_call)


            tool_exec = tool_list[tool_name]

            tool_result = tool_exec.invoke(tool_call)
            messages.append(tool_result)
            # Tool Message 추가
        tool_msg = llm_with_tools.invoke(messages)
        # AIMessage
        print('## LLM 호출 ##')
        messages.append(tool_msg)

        if tool_msg.content:
            print('LLM:', tool_msg.content)


    result = tool_msg

    return messages

react_agent(llm, msgs, tools)

NameError: name 'question' is not defined

LangGraph의 prebuilt를 이용하면 이전의 코드를 매우 간결하게 구성할 수 있습니다.

In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
agent = create_react_agent(llm, tools)
# system_prompt를 통해 시스템 프롬프트도 커스텀

In [ ]:
agent.invoke(
    {'messages':[HumanMessage('파일의 위치가 ./sample_data.csv 이다. 이 파일의 제품별 판매량을 비교하여 result.png에 시각화해줘. 묻지 말고 끝까지 해.')]})